In [ ]:
import numpy as np
from astropy.timeseries import LombScargle
import matplotlib.pyplot as plt
import lightkurve as lk 
import pandas as pd

In [ ]:
df_tess = pd.read_hdf("../data/tess_data_HD38529.h5")


# read in clean data 

time = np.array(df_tess.time.values)
flux = np.array(df_tess.flux.values)

good = np.logical_and(np.isfinite(time), np.isfinite(flux))

time = time[good] - np.median(time[good])
flux = flux[good] - np.median(flux[good])


In [ ]:
# plot data 

fig, ax = plt.subplots(figsize = (14,7))

plt.scatter(time, flux, s = 1)

plt.title("TESS")
plt.xlabel("Time (days)")
plt.ylabel("Normalized flux")
plt.show()

In [ ]:
# This set of functions does the traditional fitting Hogg loves

def _hogg_design_matrix(fs, ts):
    assert np.all(fs > 0.)
    N = len(ts)
    X = np.ones_like(ts)
    for f in fs:
        X = np.vstack([np.exp(-2.j * np.pi * f * ts), X, np.exp(2.j * np.pi * f * ts)])
    return X.T

def hogg_traditional_fit(ts, ys, fs):
    X = _hogg_design_matrix(fs, ts)
    Zs, _, _, _ = np.linalg.lstsq(X, ys, rcond=1e-9) # MAGIC
    return Zs

def hogg_traditional_synthesize(fs, Zs, ts):
    return _hogg_design_matrix(fs, ts) @ Zs


In [ ]:
def WtLSP_init(time):
    fpeaks = np.array([])
    ppeaks = np.array([])
    deltaf = 0.5/(np.max(time) - np.min(time))
    maxf = 0.5 / np.median(time[1:] - time[:-1]) # assumes data are ordered in time
    fgrid = np.arange(deltaf, maxf, deltaf)
    
    return fpeaks, ppeaks, deltaf, fgrid

In [ ]:
def parabola_peak(ys, plot= False):
    """
    ## Notes:
    - Assumes `ys` is shape `(3, )`
    - Assumes data are equally spaced!
    """
    y_minus, y_zero, y_plus = ys
    a0 = y_zero
    a1 = (y_plus - y_minus) / 2
    a2 = y_plus - (2. * y_zero) + y_minus
    x_max = -1. * a1 / a2
    y_max = a0 + a1 * x_max + 0.5 * a2 * x_max * x_max
    
    if plot: 
        print (a0, a1, a2)
        plt.scatter(np.arange(3) - 1, ys)
        xplot = np.linspace(-1.5,1.5,100)
        
        plt.plot(xplot,a0 + a1 * xplot + 0.5 * a2 * xplot**2, 'r-')
        plt.scatter(x_max, y_max)
        
        #plt.scatter(fgrid[maxi - 1: maxi + 2], pgrid[maxi - 1: maxi + 2])

    return x_max, y_max

In [ ]:
def create_fs_horror(fpeaks, deltaf, Khalf=1):
    tiny = 1.e-9 # magic
    foo = np.concatenate([np.arange(f - Khalf * deltaf, f + (Khalf + 0.5) * deltaf, deltaf) for f in fpeaks])
    return foo[foo > tiny]

def WtLSP_step(time, flux, resid, fps, pps, deltaf, fgrid, i, maxiter, ax=None):
    
    #print (time.shape, resid.shape)
    pgrid = LombScargle(time, resid).power(fgrid)
    
    # find the tallest peak 
    maxi = np.argmax(pgrid)
    
    if maxi == 0:
        fpeaks = np.append(fps, fgrid[0])
        ppeaks = np.append(pps, pgrid[0])
    else: 
        dimensionless_shift, p = parabola_peak(pgrid[maxi - 1: maxi + 2])
        fpeaks = np.append(fps, fgrid[maxi] + dimensionless_shift * deltaf)
        ppeaks = np.append(pps, p)

    fs = create_fs_horror(fpeaks, deltaf)
    Zs = hogg_traditional_fit(time, flux, fs)
    resid = flux - hogg_traditional_synthesize(fs, Zs, time).real

    if ax is not None: 
        # if True:                         # make all plots
        if (i == 0) or (i == maxiter - 1): # make only the first and last plot
            WtLSP_plot(ax, fgrid, pgrid, i)
        
    return resid, fpeaks, ppeaks

In [ ]:
def WtLSP_plot(ax, fgrid, pgrid, i):
    ax.plot(fgrid / 0.0864, pgrid, label = str(i), alpha = 0.5) 

In [ ]:
def WtLSP(time, flux, maxiter=5):

    fpeaks, ppeaks, deltaf, fgrid = WtLSP_init(time)
        
    resid = flux.copy()
    
    fig, ax = plt.subplots(figsize = (14,7))
    
    for i in range(maxiter):
        
        resid, fpeaks, ppeaks = WtLSP_step(time, flux, resid, fpeaks, ppeaks,
                                           deltaf, fgrid, i, maxiter, ax=ax)
        print(ppeaks)
    
    ax.loglog()
    ax.legend()
    ax.set_ylim(1.e-4 * np.max(ppeaks), 1.e1 * np.max(ppeaks))
    return resid, fpeaks, ppeaks, ax

In [ ]:
resid, fpeaks, ppeaks, ax = WtLSP(time, flux, maxiter=15)
ax.scatter(fpeaks, ppeaks)

In [ ]:
fig, ax = plt.subplots(figsize = (14,7))
plt.scatter(time, flux, s= 1)
plt.scatter(time, resid, s= 1)